# Part 3 - Pairwise Alignment

In this notebook we will briefly show you how to perform pairwise alignments with Biopython and how to work with some alignment files. Note that we're only performing pairwise alignments here, using local and global approaches. Multiple sequence alignments will be covered later in the course.

In [ ]:
# install and/or load BioPython
%pip install biopython

# replace this with your e-mail address
EMAIL = 'A.N.Other@somewhere.com'

In [5]:
from Bio import Entrez
from Bio import SeqIO
from Bio import pairwise2 as pw
from Bio import AlignIO
from Bio import Align as al

# list the available scoring matrices from the SubsMat module
print(al.substitution_matrices.load())

Entrez.email = EMAIL

# the accession ids of human beta-globin and myoglobin proteins respectively
protein_ids = ['NP_000509.1','NP_005359.1']

handle = Entrez.efetch(db="protein", id=protein_ids, rettype='fasta',retmode="text")
records = list(SeqIO.parse(handle, "fasta"))
handle.close()

# use these as sequence objects
beta_globin = records[0].seq
myoglobin = records[1].seq

['BENNER22', 'BENNER6', 'BENNER74', 'BLOSUM45', 'BLOSUM50', 'BLOSUM62', 'BLOSUM80', 'BLOSUM90', 'DAYHOFF', 'FENG', 'GENETIC', 'GONNET1992', 'HOXD70', 'JOHNSON', 'JONES', 'LEVIN', 'MCLACHLAN', 'MDM78', 'NUC.4.4', 'PAM250', 'PAM30', 'PAM70', 'RAO', 'RISLER', 'SCHNEIDER', 'STR', 'TRANS']


The [pairwise2 module](https://biopython.org/docs/dev/api/Bio.pairwise2.html) has two main functions for alignment 'local' and 'global' when they are called you add two charcters on to those to define how you want to perform the search, for example globalxx or localxx. Those two letters define the following:-

```
CODE  DESCRIPTION
x     No parameters. Identical characters have score of 1, otherwise 0.
m     A match score is the score of identical chars, otherwise mismatch
      score.
d     A dictionary returns the score of any pair of characters.
c     A callback function returns scores.
The gap penalty parameters are:

CODE  DESCRIPTION
x     No gap penalties.
s     Same open and extend gap penalties for both sequences.
d     The sequences have different open and extend gap penalties.
c     A callback function returns the gap penalties.
```

Further details can be found [here](https://biopython.org/DIST/docs/api/Bio.pairwise2-module.html).

As an example though a call of :-
```
pairwise2.align.globalms("ACCGT", "ACG", 2, -2, -.5, -.1)
```

for 'm' you specify match (+2) and mismatch (-2) scores and then 's' you specify gap-open (-0.5) and gap-extend (-0.1) scores

In [7]:
# perform a pairwise local alignment using the pam250 substitution matrix
mx = al.substitution_matrices.load('PAM250')

alignments = pw.align.localds(beta_globin,myoglobin,mx, -10, -0.5)

# this tells us how many alignments have the same optimal score (pretty useful, think of cells with more than 
# one backtrace mark in the hand-drawn alignments)
print(len(alignments))

# in the result we can extract several score features

# the alignment score
print(alignments[0][2])

# the start of the alignment
print(alignments[0][3])

# the end of the alignment
print(alignments[0][4])

# note here we are using 'd' the pam250 scoring matrix and then 's' gap-open (-10) and gap-extend (-0.5)

# unfortunately pairwise2 output looks awful (but its a good built in alignment method for you to practice
# with sequence alignment matrices and scoring systems

# so we're going to make a very basic fasta format file and use AlignIO to convert it into Clustal alignment
# format which is much nicer to look at

#create the fasta format
# > aligned seq 1
# SEQUENCE
# > aligned seq 2
# SEQUENCE

alignment_fasta = \
">"+records[0].name+" "+records[0].description+"\n"+alignments[0][0] \
+"\n"+ \
">"+records[1].name+" "+records[1].description+"\n"+alignments[0][1]

# write it to a file
fh = open('data/globin_alignment_pam250.fa','w')
fh.write(alignment_fasta)
fh.close()

# read in the file using AlignIO
alignment = AlignIO.read("data/globin_alignment_pam250.fa", "fasta")

# convert to clustal
print(format(alignment,'clustal'))

2
172.5
3
148
CLUSTAL X (1.81) multiple sequence alignment


NP_000509.1                         MVHLTPEEKSAVTALWGKV--NVDEVGGEALGRLLVVYPWTQRFFESFGD
NP_005359.1                         -MGLSDGEWQLVLNVWGKVEADIPGHGQEVLIRLFKGHPETLEKFDKFKH

NP_000509.1                         LSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLH
NP_005359.1                         LKSEDEMKASEDLKKHGATVLTALGGILKKKGHHEAEIKPLAQSHATKHK

NP_000509.1                         VDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH-
NP_005359.1                         IPVKYLEFISECIIQVLQSKHPGDFGADAQGAMNKALELFRKDMASNYKE

NP_000509.1                         -----
NP_005359.1                         LGFQG





In [ ]:
# perform a pairwise local alignment using the pam10 substitution matrix
mx = al.substitution_matrices.load('PAM30')

alignments = pw.align.localds(beta_globin,myoglobin,pam30, -10, -0.5)

alignment_fasta = \
">"+records[0].name+" "+records[0].description+"\n"+alignments[0][0] \
+"\n"+ \
">"+records[1].name+" "+records[1].description+"\n"+alignments[0][1]

# write it to a file
fh = open('data/globin_alignment_pam30.fa','w')
fh.write(alignment_fasta)
fh.close()

# read in the file using AlignIO
alignment = AlignIO.read("data/globin_alignment_pam30.fa", "fasta")

# convert to clustal
print(format(alignment,'clustal'))

In [ ]:
# perform a pairwise global alignment using the pam250 substitution matrix
mx = al.substitution_matrices.load('PAM250')

alignments = pw.align.globalds(beta_globin,myoglobin,pam250, -10, -0.5)

# this tells us how many alignments have the same optimal score (pretty useful, think of cells with more than 
# one backtrace mark in the hand-drawn alignments)
print(len(alignments))

# in the result we can extract several score features

# the alignment score
print(alignments[0][2])

# the start of the alignment (NB global alignments must always start at 0)
print(alignments[0][3])

# the end of the alignment
print(alignments[0][4])

alignment_fasta = \
">"+records[0].name+" "+records[0].description+"\n"+alignments[0][0] \
+"\n"+ \
">"+records[1].name+" "+records[1].description+"\n"+alignments[0][1]

# write it to a file
fh = open('data/globin_alignment_pam250_global.fa','w')
fh.write(alignment_fasta)
fh.close()

# read in the file using AlignIO
alignment = AlignIO.read("data/globin_alignment_pam250_global.fa", "fasta")

# convert to clustal
print(format(alignment,'clustal'))

### Challenge - Practice Experiments
By modifying the code above try pulling different kinds of sequences to align using both local and alignment methods. Play with the choice of alignment matrix and different parameters to see what effects they have on the alignments generated.